In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from scipy.stats import norm
import pandas as pd
import numpy as np

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import *



In [2]:
def calculate_target_risk_std(adjusted_close: pd.Series) -> float:
    """
    Calculates the annualized standard deviation (volatility) of log returns
    over the most recent 30 business days.
    """
    daily_returns = np.log(adjusted_close).diff()
    std = daily_returns.tail(30).std()
    return std * np.sqrt(BUSINESS_DAYS_IN_YEAR)


def calculate_fixed_risk_position(capital: float,
                                  target_risk_tau: float,
                                  price: pd.Series,
                                  fx: pd.Series,
                                  multiplier: float,
                                  ann_volatility: float) -> pd.Series:
    """
    Calculates the number of contracts for a fixed target risk level (τ).

    Formula:
        N = (Capital x τ) / (Multiplier x Price x FX x std%)

    where:
        τ = target risk fraction (e.g. 0.10 for 10%)
        std% = annualized volatility (in decimal)
    """
    position_contracts = capital * target_risk_tau / (multiplier * price * fx * ann_volatility)
    return position_contracts


def calculate_minimum_capital(multiplier: float,
                              price: float,
                              fx: float,
                              ann_volatility: float,
                              target_risk: float,
                              n_contracts: int = 4) -> float:
    """
    Calculates the minimum capital required to reach a given target annual risk.

    Formula:
        Capital_min = (N x Multiplier x Price x FX x std%) / τ
    """
    min_capital = n_contracts * multiplier * price * fx * ann_volatility / target_risk
    return min_capital



In [3]:
DATA_DIR = '../_databases'
df_win = pd.read_excel((DATA_DIR + '/WINFUT.xlsx'),decimal=',',index_col='date')
df_win.sort_index(inplace=True)

df_wdo = pd.read_excel((DATA_DIR + '/WDOFUT.xlsx'),decimal=',',index_col='date')
df_wdo.sort_index(inplace=True)

In [ ]:
adjusted_price = df_wdo["adjusted_close"]
current_price = df_wdo["close"]
multiplier = 10
risk_target_tau = 0.1
fx_series = pd.Series(1, index=df_wdo.index)  # FX rate, 1 for USD/USD

capital = 100000

# --- Calculate instrument annualized risk (σ%)
instrument_risk = calculate_target_risk_std(adjusted_close=adjusted_price)

# --- Calculate position sizing for fixed target risk
position_contracts_held = calculate_fixed_risk_position(
    capital=capital,
    fx=fx_series,
    ann_volatility=instrument_risk,
    target_risk_tau=risk_target_tau,
    multiplier=multiplier,
    price=current_price,
)

# --- Compute returns
perc_return = calculate_perc_returns(
    position_contracts_held=position_contracts_held,
    adjusted_price=adjusted_price,
    fx=fx_series,
    capital=capital,
    multiplier=multiplier,
)


# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=perc_return.index,
        y=perc_return.cumsum()*100,
        name="WDOFUT - Estratégia com ajuste de risco",
        line=dict(color="blue"),
    )
)

fig.update_layout(
    title="Estratégia com Ajuste de Risco - Retorno Acumulado",
    xaxis_title="Data",
    yaxis_title="Retorno Acumulado (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho - WDO")
print("Estatísticas na Frequência Diária")
print(calculate_stats(perc_return))
print("================================")
print("Estatísticas na Frequência Mensal")
print(calculate_stats(perc_return, freq=MONTH))
print("================================")
print("Capital Mínimo Necessário para 50 Contratos de WDO")
print(
    calculate_minimum_capital(
        multiplier=multiplier,
        target_risk=risk_target_tau,
        fx=1,
        ann_volatility=instrument_risk,
        price=current_price.iloc[-1],
        n_contracts=50,
    )
)

Estatísticas de Desempenho - WDO
Estatísticas na Frequência Diária
{'ann_mean': -0.08129369451096284, 'ann_std': 0.3254738951822806, 'sharpe': -0.24977024490838065, 'skew': 0.5349759840355126, 'avg_drawdown': 1.6598130733437197, 'max_drawdown': 2.4499514208036164, 'quant_lower': 1.9814859984238182, 'quant_upper': 2.2776189640456725}
Estatísticas na Frequência Mensal
{'ann_mean': -1.675112753967767, 'ann_std': 1.3459748934414932, 'sharpe': -1.2445349182440606, 'skew': 0.7306880403187761, 'avg_drawdown': 1.5436907899924233, 'max_drawdown': 2.3136988092614073, 'quant_lower': 1.536509338421612, 'quant_upper': 2.342691901449511}
Capital Mínimo Necessário para 50 Contratos de WDO
3298673.5318404934


In [ ]:
adjusted_price = df_win["adjusted_close"]
current_price = df_win["close"]
multiplier = 0.2
risk_target_tau = 0.25
fx_series = pd.Series(1, index=df_win.index)  # FX rate, 1 for USD/USD ou BRL/BRL

capital = 100000

# --- Calculate instrument annualized risk (σ%)
instrument_risk = calculate_target_risk_std(adjusted_close=adjusted_price)

# --- Calculate position sizing for fixed target risk
position_contracts_held = calculate_fixed_risk_position(
    capital=capital,
    fx=fx_series,
    ann_volatility=instrument_risk,
    target_risk_tau=risk_target_tau,
    multiplier=multiplier,
    price=current_price,
)

# --- Compute returns
perc_return = calculate_perc_returns(
    position_contracts_held=position_contracts_held,
    adjusted_price=adjusted_price,
    fx=fx_series,
    capital=capital,
    multiplier=multiplier,
)


# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=perc_return.index,
        y=perc_return.cumsum()*100,
        name="WINFUT - Estratégia com ajuste de risco",
        line=dict(color="blue"),
    )
)

fig.update_layout(
    title="Estratégia com Ajuste de Risco - alvo std 10%- Retorno Acumulado",
    xaxis_title="Data",
    yaxis_title="Retorno Acumulado (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho - WIN")
print("Estatísticas na Frequência Diária")
print(calculate_stats(perc_return))
print("================================")
print("Estatísticas na Frequência Mensal")
print(calculate_stats(perc_return, freq=MONTH))
print("================================")
print("Capital Mínimo Necessário para 50 Contratos de WIN")
print(
    calculate_minimum_capital(
        multiplier=multiplier,
        target_risk=risk_target_tau,
        fx=1,
        ann_volatility=instrument_risk,
        price=current_price.iloc[-1],
        n_contracts=100,
    )
)


Estatísticas de Desempenho - WIN
Estatísticas na Frequência Diária
{'ann_mean': 0.3474925751402522, 'ann_std': 1.7307520733122919, 'sharpe': 0.20077547818574917, 'skew': 0.1720516598023681, 'avg_drawdown': 2.3394900022809413, 'max_drawdown': 7.149088532361822, 'quant_lower': 2.4685821905753205, 'quant_upper': 2.254410449645766}
Estatísticas na Frequência Mensal
{'ann_mean': 7.16314979079764, 'ann_std': 5.961284524816909, 'sharpe': 1.2016117937295812, 'skew': -0.3494452335641036, 'avg_drawdown': 2.179496037448317, 'max_drawdown': 5.1488758100624485, 'quant_lower': 1.5393003448555196, 'quant_upper': 1.7057199972732695}
Capital Mínimo Necessário para 50 Contratos de WIN
1406695.5134464311
